 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [729]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
import scipy.sparse as sp

##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number



Pandas version: 1.4.2
Numpy version: 1.21.5


In [730]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [731]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [732]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False


In [733]:
if import_mode: 
    ##Change working directory to temp/epic
    os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


Current Working Directory is:  C:\temp\epic


In [734]:
if import_mode:
    print(os.listdir(cwd)) #List directory

['OLD_PID.csv', 'PID.xlsx', 'TND.xlsx']


#Random_mode:
Generate aleatories PID numbers and symbols.



In [735]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [736]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

Series and DataFrames

In [737]:
#pid_series = pd.Series(random_PID_symbols, random_PID_numbers)
#pid_series = pid_series.sort_index(axis=0,ascending=True)
#pid_series

#Import Tag number definitions as a pandas series
#tnd = pd.read_csv('TND.csv',names=['PID','Description'])
#tnd = pd.read_excel('TND.xlsx',)
#tnd

In [738]:
#PID_series = tnd['PID'].squeeze()
#pid_exist = PID_series.str.contains('.100')

In [739]:
#tnd.at[0,'Description']

In [740]:
tnd_pid = pd.read_excel('TND.xlsx',index_col='PID')
tnd_pid

,Description
PID,
1.100,Intake air fan
1.101,Filter 1
1.102,Filter 2
1.103,Filter 3
1.104,NaN
...,...
1.995,NaN
1.996,NaN
1.997,NaN


In [741]:
tnd_pid.at[1.100,'Description']

'Intake air fan'

Reading Excel File

We can read an excel file into a data frame:

pd.read_excel('myExcel.xlsx', index_col=['ColumnA'])

We can also load one sheet into a data frame:

pd.read_excel(open('myExcel.xlsx', 'rb'), sheet_name='Sheet1')

In [742]:
#df_data_excel = pd.read_excel('PID.xlsx', index_col='Prop_PID_Number')
df_data_excel = pd.read_excel('PID.xlsx')

In [743]:
#Remove ShapeKey from the dataframe
del df_data_excel['ShapeKey']

In [744]:
df_data_excel.count()

Prop_PID_Element    289
Prop_PID_Number     289
dtype: int64

In [745]:
#Remove duplicates - It removes duplicates alarms as 1.310 Particle counter 0.5 um and 5.0um

df_data_excel_uniques = df_data_excel.drop_duplicates()

In [746]:
df_data_excel_uniques.count()

Prop_PID_Element    281
Prop_PID_Number     281
dtype: int64

In [747]:
gkk = df_data_excel_uniques.groupby(['Prop_PID_Number', 'Prop_PID_Element'])
gkk.first()

Empty DataFrame
Columns: []
Index: [(1.1, M), (1.1, SC), (1.1, V), (1.1, XA), (1.101, F), (1.102, F), (1.108, PDI), (1.111, F), (1.112, F), (1.15, M), (1.15, SC), (1.15, TSA+), (1.15, V), (1.151, F), (1.152, F), (1.155, C), (1.158, PDI), (1.16, M), (1.16, SC), (1.16, V), (1.16, XA), (1.2, M), (1.2, SC), (1.2, V), (1.2, XA), (1.201, F), (1.208, PDI), (1.21, M), (1.21, SC), (1.21, V), (1.21, XA), (1.211, F), (1.218, PDI), (1.221, F), (1.232, F), (1.24, M), (1.24, SC), (1.24, V), (1.24, XA), (1.241, F), (1.242, F), (1.248, PDI), (1.3, FA-), (1.3, QIA+), (1.3, XA), (1.304, M), (1.304, P), (1.305, GSA+), (1.305, GSA-), (1.305, V), (1.305, Y), (1.308, FSA-), (1.31, FA-), (1.31, QIA+), (1.31, XA), (1.314, M), (1.314, P), (1.315, GSA+), (1.315, GSA-), (1.315, V), (1.315, Y), (1.318, FSA-), (1.32, FA-), (1.32, QIA+), (1.32, XA), (1.324, M), (1.324, P), (1.325, GSA+), (1.325, GSA-), (1.325, V), (1.325, Y), (1.328, FSA-), (1.33, FA-), (1.33, QIA+), (1.33, XA), (1.334, M), (1.334, P), (1.335, GSA+), (1.335, GSA-), (1.335, V), (1.335, Y), (1.338, FSA-), (1.35, P), (1.35, XA), (1.36, P), (1.36, XA), (1.37, P), (1.37, XA), (1.4, GIA), (1.4, M), (1.4, SC), (1.4, XA), (1.41, GIA), (1.41, M), (1.41, SC), (1.41, XA), (1.52, GSA+), (1.52, GSA-), (1.52, K), (1.52, Y), ...]

[281 rows x 0 columns]

In [748]:
#Get uniques values from the PID Elements (the columns for the matrix)
df_unique_elememts = df_data_excel.Prop_PID_Element.unique()
df_unique_elememts.sort()
print(df_unique_elememts)




['B' 'C' 'D' 'EC' 'F' 'FA-' 'FIA+' 'FICA±' 'FSA-' 'GIA' 'GSA+' 'GSA-'
 'GZA-' 'H' 'K' 'L' 'LSA-' 'M' 'MIA±' 'NC' 'P' 'PDI' 'PDICA±' 'PDIS' 'PI'
 'PSA-' 'QIA+' 'QIA-' 'QSA+' 'S' 'SC' 'SIA±' 'T' 'TIC' 'TICA±' 'TSA+' 'V'
 'W' 'WICA±' 'XA' 'Y' 'YC']


In [749]:
#Sort by PID number
df_data_excel_sorted = df_data_excel_uniques.sort_values(by='Prop_PID_Number')

In [750]:
df_data_excel_sorted.head(1000)

,Prop_PID_Element,Prop_PID_Number
209,V,1.100
251,XA,1.100
186,SC,1.100
107,M,1.100
4,F,1.101
...,...,...
130,M,2.304
46,FSA-,2.308
270,XA,2.350
150,P,2.350
